<a href="https://colab.research.google.com/github/ashishshinde0901/MesoWest_HRRR_weather_data_exctraction/blob/main/weather_data_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
lat = 40.7608
longi = -111.8910
start = datetime.datetime.strptime("20230721_00", '%Y%m%d_%H')
forecast_length = 3936

First Index: 644
Second Index: 495


In [13]:
%%time
import datetime
import numpy as np
import pandas as pd
import s3fs
import zarr
import xarray as xr
import s3fs
import cartopy as ccrs

# Initialize the S3 filesystem and load the chunk index
fs = s3fs.S3FileSystem(anon=True)
chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
projection = ccrs.crs.LambertConformal(central_longitude=-97.5,
                                        central_latitude=38.5,
                                        standard_parallels=[38.5])

def get_nearest_indices(projection, chunk_index, latitude, longitude):
    """
    Get the nearest indices in the chunk index for a given latitude and longitude.

    Parameters:
    - projection: Cartopy projection object
    - chunk_index: Xarray dataset with chunk indices
    - latitude: Latitude of the point
    - longitude: Longitude of the point

    Returns:
    - first_index: The index_y value from the nearest point
    - second_index: The index_x value from the nearest point
    """
    x, y = projection.transform_point(longitude, latitude, ccrs.crs.PlateCarree())
    nearest_point = chunk_index.sel(x=x, y=y, method="nearest")

    first_index = int(nearest_point.index_y.values)
    second_index = int(nearest_point.index_x.values)

    return first_index, second_index



first_index, second_index = get_nearest_indices(projection, chunk_index, lat, longi)

print("First Index:", first_index)
print("Second Index:", second_index)




arr = pd.DataFrame()
tmp_analysis = []
i = 0

# data = np.empty(forecast_length, dtype=[('Analysis Time', 'datetime64[s]'), ('TMP', 'float')])

for time_delta in range(forecast_length):
    # construct the aws path
    analysis_time = start + datetime.timedelta(hours=time_delta)
    # print(analysis_time)


    string_date = analysis_time.strftime('%Y%m%d')
    string_hour   = analysis_time.strftime('%H')

    url = 's3://hrrrzarr/sfc/%s/%s_%sz_anl.zarr' %(string_date, string_date, string_hour)
    # url = 's3://hrrrzarr/sfc/%s/%s_%sz_anl.zarr' %(string_date, string_date, string_hour)

    try:
        store = zarr.open(s3fs.S3Map(url, s3=fs))
        print(f"Successfully opened {url}")


        temp = store["surface/TMP/surface/TMP"][first_index, second_index]
        smoke = store["8m_above_ground/MASSDEN/8m_above_ground/MASSDEN"][first_index, second_index]
        Vertically_integrated_smoke = store["entire_atmosphere_single_layer/COLMD/entire_atmosphere_single_layer/COLMD"][first_index, second_index]
        dew_point = store["2m_above_ground/DPT/2m_above_ground/DPT"][first_index, second_index]
       	wind_speed = store["10m_above_ground/WIND_max_fcst/10m_above_ground/WIND_max_fcst"][first_index, second_index]
        relative_humidity = store["2m_above_ground/RH/2m_above_ground/RH"][first_index, second_index]
        specific_humidity = store["2m_above_ground/SPFH/2m_above_ground/SPFH"][first_index, second_index]
        # pres_val = store["surface/PRES/surface/PRES"][first_index, second_index]
        # #APCP_fcst = store["surface/APCP_acc_fcst/surface/APCP_acc_fcst"][first_index, second_index]
        # cape = store["surface/CAPE/surface/CAPE"][first_index, second_index]
        # cin = store["surface/CIN/surface/CIN"][first_index, second_index]
        # cnwat = store["surface/CNWAT/surface/CNWAT"][first_index, second_index]
        dlwrf = store["surface/DLWRF/surface/DLWRF"][first_index, second_index]
        dswrf = store["surface/DSWRF/surface/DSWRF"][first_index, second_index]
        # fricv = store["surface/FRICV/surface/FRICV"][first_index, second_index]
        gflux = store["surface/GFLUX/surface/GFLUX"][first_index, second_index]
        gust = store["surface/GUST/surface/GUST"][first_index, second_index]
        hgt = store["surface/HGT/surface/HGT"][first_index, second_index]
        # hpbl = store["surface/HPBL/surface/HPBL"][first_index, second_index]
        # icec = store["surface/ICEC/surface/ICEC"][first_index, second_index]
        # land = store["surface/LAND/surface/LAND"][first_index, second_index]
        # lhtfl = store["surface/LHTFL/surface/LHTFL"][first_index, second_index]
        # sfcr = store["surface/SFCR/surface/SFCR"][first_index, second_index]
        # shtfl = store["surface/SHTFL/surface/SHTFL"][first_index, second_index]
        # snod = store["surface/SNOD/surface/SNOD"][first_index, second_index]
        # snowc = store["surface/SNOWC/surface/SNOWC"][first_index, second_index]
        ulwrf = store["surface/ULWRF/surface/ULWRF"][first_index, second_index]
        uswrf = store["surface/USWRF/surface/USWRF"][first_index, second_index]
        # vbdsf = store["surface/VBDSF/surface/VBDSF"][first_index, second_index]
        # vddsf = store["surface/VDDSF/surface/VDDSF"][first_index, second_index]
        # vgtyp = store["surface/VGTYP/surface/VGTYP"][first_index, second_index]
        # vis = store["surface/VIS/surface/VIS"][first_index, second_index]
        # weasd= store["surface/WEASD/surface/WEASD"][first_index, second_index]







        arr.at[analysis_time,"Temprature"] = temp
        arr.at[analysis_time,"Smoke_8m_above_ground"] = smoke
        arr.at[analysis_time,"Vertically Integrated Smoke"] = Vertically_integrated_smoke
        arr.at[analysis_time,"Dew Point"] = dew_point
        arr.at[analysis_time,"Wind Speed"] = wind_speed
        arr.at[analysis_time,"Relative Humidity"] = relative_humidity
        arr.at[analysis_time,"Specific Humidity"] = specific_humidity
        #arr.at[analysis_time,"Surface Pressure"] = pres_val
        #arr.at[analysis_time,"Convective Available Potential Energy"] = cape
        # arr.at[analysis_time,"Convective Inhibition"] = cin
        # arr.at[analysis_time,"Plant Canopy Surface Water"] = cnwat
        arr.at[analysis_time,"Downward Long-Wave Rad. Flux"] = dlwrf
        arr.at[analysis_time,"Downward Short-Wave Rad. Flux"] = dswrf
        # arr.at[analysis_time,"Frictional Velocity"] = fricv
        arr.at[analysis_time,"Ground Heat Flux"] = gflux
        arr.at[analysis_time,"Wind Gust "] = gust
        arr.at[analysis_time,"Height of the Level of Free Convection"] =  hgt
        # arr.at[analysis_time,"Planetary Boundary Layer Height"] = hpbl
        # arr.at[analysis_time,"Ice Cover"] = icec
        # arr.at[analysis_time,"Land Cover (0=sea, 1=land)"] = land
        # arr.at[analysis_time,"Latent Heat Net Flux"] = lhtfl
        # arr.at[analysis_time,"Surface Roughness "] = sfcr
        # arr.at[analysis_time,"Sensible Heat Net Flux"] = shtfl
        # arr.at[analysis_time,"Snow Depth"] = snod
        # arr.at[analysis_time,"Snow Cover"] = snowc
        arr.at[analysis_time,"Upward Long-Wave Rad. Flux "] = ulwrf
        arr.at[analysis_time,"Upward Short-Wave Rad. Flux"] = uswrf
        # arr.at[analysis_time,"Visible Beam Downward Solar Flux "] = vbdsf
        # arr.at[analysis_time,"Visible Diffuse Downward Solar Flux"] = vddsf
        # arr.at[analysis_time,"Vegetation Type"] = vgtyp
        # arr.at[analysis_time,"Visibility "] = vis
        # arr.at[analysis_time,"Water Equivalent of Accumulated Snow Depth"] = weasd





    # append the data we need
        tmp_analysis.append(store["surface/TMP/surface/TMP"][first_index, second_index])
        # Do your processing here
    except Exception as e:
        print(f"Failed to open {url}: {e}")
        # Continue with the next iteration if an error occurs
        continue


Successfully opened s3://hrrrzarr/sfc/20230721/20230721_00z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_01z_anl.zarr


KeyboardInterrupt: 

In [11]:
pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 55.4 MB/s eta 0:00:00


In [6]:
pip install zarr

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 64.9 MB/s eta 0:00:00
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5033 sha256=0814f0199e37795b79e3cfc560c39c84439374dbecb8fc14b0aa2637c1b13e10
  Stored in directory: /root/.cache/pip/wheels/7f/4e/be/1171b40f43b918087657ec57cf3b81fa1a2e027d8755baa184
Successfully built asciitree
